In [ ]:
!pip install langchain
!pip install pandas
!pip install vectordb2

In [ ]:
import pandas as pd
import re

In [ ]:
import unicodedata

def remove_acentos(texto):
  nfkd = unicodedata.normalize('NFKD', texto)
  palavraSemAcento = "".join([c for c in nfkd if not unicodedata.combining(c)])
  return palavraSemAcento

def tratar_texto(texto):
    texto_sem_acentos = remove_acentos(texto)
    return re.sub(r'([a-z])([A-Z])', r'\1 \2', texto_sem_acentos)

In [ ]:
dataframe = pd.read_csv("./top_musicas.csv")

In [ ]:
from langchain.schema import Document

sections = [
    Document(
        page_content= tratar_texto(row["lyrics"]),
        metadata={
            "autor": row["author"],
            "titulo": row["music"],
            "letra": row["lyrics"]
        }
    )
    for index, row in dataframe.iterrows()
]

In [ ]:
from vectordb import Memory

memory = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 14, "overlap": 10})

In [ ]:
for i in range(0, len(sections)):
  letra = sections[i].page_content

  metadata = {
    "autor": sections[i].metadata["autor"],
    "musica": sections[i].metadata["titulo"],
    "letra": sections[i].metadata["letra"]
  }

  memory.save(letra, metadata)

In [ ]:
top_musicas = memory.search("For the way I hurt, yeah", top_n=4)

def corrigir_musica(texto):
    texto_sem_acentos = remove_acentos(texto)
    return re.sub(r'([a-z])([A-Z])', r'\1\n\2', texto_sem_acentos)

for top_musica in top_musicas[:4]:
  #print(f"Com distância de: {top_musica['distance']}")
  print(top_musica['metadata']['autor']+" - "+top_musica['metadata']['musica'])
  #print(f"\n{corrigir_musica(top_musica['metadata']['letra'])}\n\n")